In [2]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [3]:
conf = SparkConf()
sc = SparkContext(conf=conf)

In [4]:
spark = SparkSession \
    .builder \
    .appName("CSV with conditions") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

#????????? não percebi bem se o config é preciso em alguns não aparece

In [5]:
df = spark.read.csv("dataset/conditions.csv") .limit(40)

In [6]:
df.printSchema()
df.show()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)

+----------+----------+--------------------+--------------------+---------+--------------------+
|       _c0|       _c1|                 _c2|                 _c3|      _c4|                 _c5|
+----------+----------+--------------------+--------------------+---------+--------------------+
|     START|      STOP|             PATIENT|           ENCOUNTER|     CODE|         DESCRIPTION|
|2017-01-14|2017-03-30|09e4e8cb-29c2-4ef...|88e540ab-a7d7-47d...| 65363002|        Otitis media|
|2012-09-15|2012-09-16|b0a03e8c-8d0f-424...|e89414dc-d0c6-478...|241929008|Acute allergic re...|
|2018-06-17|2018-06-24|09e4e8cb-29c2-4ef...|c14325b0-f7ec-431...|444814009|Viral sinusitis (...|
|2019-04-19|2019-09-26|09e4e8cb-29c2-4ef...|71af18ee-3157-408...| 65363002|        Otitis media|
|2019-04

In [7]:
Data_list = ["START","STOP","PATIENT","ENCOUNTER","CODE","DESCRIPTION"]
 
df = df.toDF(*Data_list)
df=df.filter(df.START !="START")
df=df.drop(*['START','STOP','ENCOUNTER','DESCRIPTION'])
df.show()

+--------------------+---------+
|             PATIENT|     CODE|
+--------------------+---------+
|09e4e8cb-29c2-4ef...| 65363002|
|b0a03e8c-8d0f-424...|241929008|
|09e4e8cb-29c2-4ef...|444814009|
|09e4e8cb-29c2-4ef...| 65363002|
|09e4e8cb-29c2-4ef...|444814009|
|09e4e8cb-29c2-4ef...| 33737001|
|b0a03e8c-8d0f-424...|444814009|
|b0a03e8c-8d0f-424...| 10509002|
|b0a03e8c-8d0f-424...|233678006|
|b0a03e8c-8d0f-424...|195662009|
|b0a03e8c-8d0f-424...|232353008|
|b0a03e8c-8d0f-424...|195662009|
|5420ae87-24c8-4ed...|446096008|
|5420ae87-24c8-4ed...|284551006|
|5420ae87-24c8-4ed...|283371005|
|5420ae87-24c8-4ed...| 72892002|
|5420ae87-24c8-4ed...|444814009|
|5420ae87-24c8-4ed...|195662009|
|bf1f30f2-27de-4b5...|162864005|
|bf1f30f2-27de-4b5...|283385000|
+--------------------+---------+
only showing top 20 rows



In [8]:
df = df.dropna()  #confirmar depois com tudo

In [9]:
df.show()

+--------------------+---------+
|             PATIENT|     CODE|
+--------------------+---------+
|09e4e8cb-29c2-4ef...| 65363002|
|b0a03e8c-8d0f-424...|241929008|
|09e4e8cb-29c2-4ef...|444814009|
|09e4e8cb-29c2-4ef...| 65363002|
|09e4e8cb-29c2-4ef...|444814009|
|09e4e8cb-29c2-4ef...| 33737001|
|b0a03e8c-8d0f-424...|444814009|
|b0a03e8c-8d0f-424...| 10509002|
|b0a03e8c-8d0f-424...|233678006|
|b0a03e8c-8d0f-424...|195662009|
|b0a03e8c-8d0f-424...|232353008|
|b0a03e8c-8d0f-424...|195662009|
|5420ae87-24c8-4ed...|446096008|
|5420ae87-24c8-4ed...|284551006|
|5420ae87-24c8-4ed...|283371005|
|5420ae87-24c8-4ed...| 72892002|
|5420ae87-24c8-4ed...|444814009|
|5420ae87-24c8-4ed...|195662009|
|bf1f30f2-27de-4b5...|162864005|
|bf1f30f2-27de-4b5...|283385000|
+--------------------+---------+
only showing top 20 rows



In [10]:
#Cada basquet não tem items repetidos, ou seja, se uma pessoa tiver a mesma doença mais que uma vez não é contabilizado aqui
#Pode ser interessante arranjar maneira de contabilizar, logo se vê....

from pyspark.sql.functions import collect_set, col, count

baskets = df.groupBy('PATIENT').agg(collect_set('CODE').alias('items'))
baskets.createOrReplaceTempView('baskets')
baskets.show(20,False)

+------------------------------------+-----------------------------------------------------------------+
|PATIENT                             |items                                                            |
+------------------------------------+-----------------------------------------------------------------+
|09e4e8cb-29c2-4ef4-86c0-a6ff0ba25d2a|[33737001, 65363002, 444814009]                                  |
|b0a03e8c-8d0f-4242-9548-40f4d294eba8|[241929008, 10509002, 233678006, 444814009, 195662009, 232353008]|
|5420ae87-24c8-4ed4-ad14-041d15aadae9|[446096008, 72892002, 283371005, 444814009, 195662009, 284551006]|
|bf1f30f2-27de-4b54-809b-f91b92949565|[162864005, 283385000, 239873007]                                |
|28a3cdb7-1db1-4148-8280-8a4e5b4f99e0|[156073000, 72892002, 19169002, 284551006]                       |
|6c248ef3-3fa2-4888-820e-d8a596bdc381|[162864005, 429007001, 410429000, 55822004, 239873007, 68496003] |
|90f0b8d0-3888-415f-8234-d68f7beab894|[307731004, 72892

In [11]:
baskets=baskets.drop('PATIENT')
baskets.show(20,False)

+-----------------------------------------------------------------+
|items                                                            |
+-----------------------------------------------------------------+
|[33737001, 65363002, 444814009]                                  |
|[241929008, 10509002, 233678006, 444814009, 195662009, 232353008]|
|[446096008, 72892002, 283371005, 444814009, 195662009, 284551006]|
|[162864005, 283385000, 239873007]                                |
|[156073000, 72892002, 19169002, 284551006]                       |
|[162864005, 429007001, 410429000, 55822004, 239873007, 68496003] |
|[307731004, 72892002, 444814009, 195662009, 44465007]            |
+-----------------------------------------------------------------+



In [12]:
RDD = baskets.rdd.map(list)
RDD.take(20)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 16.0 failed 1 times, most recent failure: Lost task 0.0 in stage 16.0 (TID 51) (DESKTOP-CG834TP.mshome.net executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:188)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:108)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:121)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:162)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apa

In [ ]:
df.select('CODE').distinct().show()

+---------+
|     CODE|
+---------+
| 65363002|
|241929008|
|444814009|
| 33737001|
| 10509002|
|233678006|
|195662009|
|232353008|
|446096008|
|284551006|
|283371005|
| 72892002|
|162864005|
+---------+

